# Shunting Yard Calculator
## Introduction

This notebook serves to more intuitively explain the code in ```main.py```, The initial python script I wrote for this exercise.  The program queries the user for a math expression to evaluate. It supports five operators: +, -, *, /, and ^. The diagram below describes the data pipeline, and the operations each function performs on the initial string to create an output:

![A diagram of the pipeline of the program](./mermaid-diagram-2023-09-27-030644.png)

- The ```tokenizer``` takes the user input string and converts it to a list of tokens. A token is either a number or an operator.
- The ```shunting_yard``` algorithm converts infix notation to postfix notation (reverse polish notation). For example, [$1 + 1$] becomes [$1$ $1$ $+$]. This removes the need for parenthesis.
- the ```rpn_evaluator``` takes an NPN expression and computes the final result.

## Code Explanation
### ```tokenizer``` Function

If we were to simply pass our user string to the shunting yard, our tokens would essentially be single characters- limiting the user's ability to input numbers higher than 9. For example, consider the expression ```100 - 5```
- Without a tokenizer, we pass ```[1, 0, 0, -, 5]``` to the shunting yard algorithm
- With a tokenizer, we can pas ```[100, -, 5]```

The tokenizer below is incomplete - it does not construct tokens with negative numbers, or floating point values, for reasons discussed in the "Problems and Reflections" section of this notebook. It goes through each character in the expression string and does the following: if the **character** is a digit, it adds it to a temporary string called ```token_builder```. If the character is an operator (What *is* an operator is defined via the ```operator_list``` parameter), the built-up characters in ```token_builder``` are appended to the ```tokenized_list``` as a token (Because operators separate numbers in mathematical expressions). The character (which is an operator) is then appended to the ```tokenized_list``` as its own token, and the ```token_builder``` is reset to an empty string, as the token is complete, and is prepared to start building another number token. If any other character is detected, it is ignored.

In [ ]:
# TODO: account for floating point numbers and negative numbers
def tokenizer(user_input: str, operator_list: list) -> list:
    token_builder = ""
    tokenized_list = []
    for char in user_input:
        if char.isdigit():
            token_builder += char
        elif char in operator_list:
            tokenized_list.append(token_builder)
            tokenized_list.append(char)
            token_builder = ""
        else:
            continue
    tokenized_list.append(token_builder)
    return tokenized_list

### ```shunting_yard``` function

[Edsger Djikstra](https://en.wikipedia.org/wiki/Edsger_W._Dijkstra) was a very influential computer scientist. Though the most famous of his algoritms relates to finding the shortest path in a graph, he had contributed many, among them being the shunting yard algorithm. [This six minute video](https://www.youtube.com/watch?v=Wz85Hiwi5MY) does an excellent job providing an intuitive explanation for the algorithm.

This algorithm transforms an algorithm from standard algebraic notation where the operators are infixed (such as $x + y$) to reverse polish notation, where the operators are postfixed ($x y +$)

Polish notation essentially treat operators as math functions, like when you see $f(x, y)$. In PN, this would be $+(x,y)$.  This is very convenient for computing, because in NPN we can eliminate parenthesis from our order of operations and still maintain mathematical integrity. Reversing it gives reverse polish notation, which postfixes the operator. This is convenient because when reversed, PN will always have two numbers before an operator - evaluating from left to right, every operator acts as the head of its own 'sub-stack', so when a evaluator sees an operator token, it can perform that operation on the previous two tokens and compress the substack down to a single value.

In [ ]:
def shunting_yard(tokenized_expression: list, operator_precedence: dict):
    output = []
    operator_stack = []

    opening_parenthesis_counter = 0
    closing_parenthesis_counter = 0

    for token in tokenized_expression:
        if token.isdigit():
            output.append(token)
        elif token == '(':
            opening_parenthesis_counter += 1
            operator_stack.append(token)
        elif token == ')':
            closing_parenthesis_counter += 1
            while operator_stack and operator_stack[-1] != "(":
                output.append(operator_stack.pop())
            operator_stack.pop()
        elif token in operator_precedence.keys():
            while operator_stack and operator_precedence[token] <= operator_precedence[operator_stack[-1]] and operator_stack[-1] != '(':
                output.append(operator_stack.pop())
            operator_stack.append(token)
        else:
            continue

    while operator_stack:
        output.append(operator_stack.pop())

    if opening_parenthesis_counter != closing_parenthesis_counter:
        print("warning: parenthesis mismatch detected")

    return output

### ```rpn_evaluate``` function

This is the most straightforward part of the program. The function iterates through a list, and when it sees an operator, it performs the operation on the previous two tokens. The implementation here is a little java-esque and not very standard python, which discourages modifying a list as its being iterated through as it can lead to unexpected behaviour, but it appears to be functional from my testing.

In [ ]:
def rpn_evaluate(expression: list):
    for token in expression[:]:
        if not (token.isdigit()):
            token_index = expression.index(token)

            if token == '+':
                expression[token_index-2] = float(expression[token_index-2]) + float(expression[token_index-1])
            if token == '-':
                expression[token_index-2] = float(expression[token_index-2]) - float(expression[token_index-1])
            if token == '*':
                expression[token_index - 2] = float(expression[token_index - 2]) * float(expression[token_index - 1])
            if token == '/':
                if float(expression[token_index - 1]) == 0:
                    print("[ERROR] ignoring division by zero, result will be incorrect")
                    print("RPN: " + str(expression))
                    continue
                expression[token_index - 2] = float(expression[token_index - 2]) / float(expression[token_index - 1])
            if token == '^':
                expression[token_index - 2] = float(expression[token_index - 2]) ** float(expression[token_index - 1])
            
            expression.pop(token_index)
            expression.pop(token_index-1)

    return expression[0]

### Running the Code

In [ ]:
# This dictionary maps each operator to its precedence in the order of operations.
# new symbols can be added seamlessly, with their definitions implemented in the RPN evaluator.
operator_to_precedence = {
    '+': 0,
    '-': 0,
    '*': 1,
    '/': 1,
    '^': 2,
    '(': 3,
    ')': 3
}

expression = input("Please input an expression to evaluate!: ")
print("[INFO] Expression: " + expression)

tokenized_expression = tokenizer(expression, list(operator_to_precedence.keys()))
print("[INFO] Tokenized Expression: " + str(tokenized_expression))

rpn = shunting_yard(tokenized_expression, operator_to_precedence)
print("[INFO] RPN: " + str(rpn))

result = rpn_evaluate(rpn)
print("result: " + str(result))

## Problems and Reflections

As mentioned earlier, one large issue with this implementation is the lack of support for negative number inputs, and operating on floating point numbers. From a software architecture standpoint, I'm not sure whether implementing this feature is best done in the tokenizer or the shunting yard function.

One solution discussed in class was to tokenize the ```'-'``` character as part of a number, e.g. ```"-10 + 5"``` --> ```['-10', '+', '5']``` . However, this causes issues in expressions such as ```"2 - 5"``` --> ```['2', '-5']```, where we now have two numbers and no operator. We could make it so that every time we parse a ```'-'``` character, we prepend an addition token, e.g. ```"2 - 5"``` --> ```"2 + -5"``` --> ```['2', '+', '-5']```, but this causes issues where the first number in a string is a negative number: ```"-10 + 5"``` --> ```['+', '-10', '+', '5']```, where the first operator is missing an operand. Doing the same but adding a zero as well (```"-10 + 5"``` --> ```['0', '+', '-10', '+', '5']```) resolves this, but creates the first problem again, where we have two operands but no operator (```"2 - 5"``` --> ```['2', '0', '+' '-5']```)

Considering that this is devolving into an endless whack-a-mole of circular issues, perhaps a better solution would be to modify the ```shunting_yard``` algorithm. We could try treating ```"-"``` as a special unary operator, and translating ```10 - 5``` as ```5 -1 * 10 +```. 

## Conclusion

This was a fun but challenging exercise, and I learned a fair bit about tokenization and the unexpected complexities that can arise from designing a tokenizer. It was also a good opportunity to refresh my knowledge of data structures and algorithms. For now though, I think I will be moving onto other projects and practice other concepts in programming. I am also open to suggestions if you have an idea for a solution!